# data processing:
## installing and importing needed python library's, and the data:

In [ ]:
import sys
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
import pandas as pd 
import numpy as np
import scipy.special


url = "https://docs.google.com/spreadsheets/d/1H-2dgSCb3EgCMEgXDOtoJ5ol9Lt1lmAeXistsCpbJ44/export?format=csv&gid=0"

df = pd.read_csv(url)
print (df)

## visualising:
### plotting the scatterplots:


In [ ]:
ax = df.plot(kind = "scatter", y = "ronde tijd 1", x = "leeftijd", title = "racetijden tegen leeftijd")
ax2 = df.plot(kind = "scatter", y = "ronde tijd 2", x = "leeftijd", ax = ax)
ax3 = df.plot(kind = "scatter", y = "ronde tijd 3", x = "leeftijd", ax = ax)
ax4 = df.plot(kind = "scatter", y = "avg tijd", x = "leeftijd", ax = ax, color = "r")
ax.set_ylabel("rondetijden (seconden)")

In [ ]:
ax = df.plot(kind = "scatter", y = "reactiesnelheid kleur", x = "leeftijd", title = "reactiesnelheden tegen leeftijd")
ax2 = df.plot(kind = "scatter", y = "reactiesnelheid geluid", x = "leeftijd", ax = ax)
ax.set_ylabel("Reactiesnelheid (milliseconden)")

In [ ]:
ax = df.plot(kind = "scatter", y = "ronde tijd 1", x = "rea gem", title = "racetijden tegen reactiesnelheid")
ax2 = df.plot(kind = "scatter", y = "ronde tijd 2", x = "rea gem", ax = ax)
ax3 = df.plot(kind = "scatter", y = "ronde tijd 3", x = "rea gem", ax = ax)
ax4 = df.plot(kind = "scatter", y = "avg tijd", x = "rea gem", ax = ax, color = "r")
ax.set_ylabel("racetijden (seconden)")
ax.set_xlabel("Reactiesnelheid (milliseconden)")

In [ ]:
# padding om te zorgen dat er geen error komt
df["X"] = [225, 250, 300, 350, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400]
df["mediaan"] = [80, 108, 93, 110, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125]

ax = df.plot(kind = "scatter", y = "ronde tijd 1", x = "rea gem", title = "racetijden tegen reactiesnelheid")
ax2 = df.plot(kind = "scatter", y = "ronde tijd 2", x = "rea gem", ax = ax)
ax3 = df.plot(kind = "scatter", y = "ronde tijd 3", x = "rea gem", ax = ax)
ax4 = df.plot(kind = "scatter", y = "avg tijd", x = "rea gem", ax = ax)
ax5 = df.plot(kind = "scatter", y = "mediaan", x = "X", ax = ax, color = "r")
ax6 = df.sort_values(by="X").plot(kind = "line", y = "mediaan", x = "X", ax = ax, color = "r")
ax.set_ylabel("racetijden (seconden)")
ax.set_xlabel("Reactiesnelheid (milliseconden)")

## the maths:
### calculating the correlation and p-value:

In [ ]:
def numberWithoutRounding(num, precision=3):
    [beforeDecimal, afterDecimal] = str(num).split('.')
    return beforeDecimal + '.' + afterDecimal[0:precision]

def corr1(df):
    mask = df.notna().to_numpy()
    corr = df.corr().to_numpy()
    n_rows, n_cols = df.shape

    # Initialize the return arrays for better performance
    length = int(n_cols * (n_cols - 1) / 2)
    correl = np.empty(length, dtype=np.float64)
    count = np.empty(length, dtype=np.uint64)

    # For 2-combination of columns, let `n` be the number of pairs whose
    # elements are all non-NaN. We will need that later to calculate the
    # p-value
    k = -1
    for i in range(n_cols):
        for j in range(i):
            n = 0
            for row in range(n_rows):
                n += 1 if mask[row, i] and mask[row, j] else 0

            k += 1
            correl[k] = corr[i,j]
            count[k] = n
            

    # The p-value can be obtained with the incomplete Beta function (betainc)
    alpha = count / 2 - 1
    x = (correl + 1) / 2
    x = np.where(correl < 0, x, 1 - x)
    p = 2 * scipy.special.betainc(alpha, alpha, x)

    confidence = numberWithoutRounding(100 - ((p[0])*100), precision= 2)
    col_a, col_b = df.columns
    print(f"Data van het vergelijken van {col_a} en {col_b}:")
    print(f'De bevonden correlatiecoëfficiënt is: {numberWithoutRounding(correl[0])}. ')
    print(f'En een bevonden p waarde van: {numberWithoutRounding(p[0])}.')
    print(f"Er is dus een {confidence}% kans dat er een correlatie is.")
    if p <= 0.05:
        print("Deze bevindingen zijn statistisch significant.")
    else:
        print("Helaas zijn de bevindingen niet statistisch significant.")
    print("")
    print("")
    

corr1(df[['reactiesnelheid geluid','reactiesnelheid kleur']])
corr1(df[['reactiesnelheid geluid','avg tijd']])
corr1(df[['reactiesnelheid kleur','avg tijd']])
corr1(df[["leeftijd", 'avg tijd']])
corr1(df[["leeftijd", "reactiesnelheid kleur"]])
corr1(df[["leeftijd", "reactiesnelheid geluid"]])


## credits:
### function to calculate p-value and correlation:
https://stackoverflow.com/questions/64682256/how-to-calculate-a-correlation-with-p-values-most-performant-in-python
by: https://stackoverflow.com/users/2538939/code-different
### rounding function:
https://stackoverflow.com/questions/60250522/how-to-show-exactly-4-numbers-after-decimal-point-without-rounding-in-python
by: https://stackoverflow.com/users/4091874/caseygrun